## Setup

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import altair as alt
from matplotlib.colors import ListedColormap

## Load and prepare data

In [ ]:
# Load the CSV file into a DataFrame
file_path = "ba-europe-countries.csv"
df = pd.read_csv(file_path)

# Filter out Europe, Belarus, Ukraine and Luxembourg (too big and too small)
df_filtered = df.drop(['Europe', 'Belarus', 'Ukraine', 'Luxembourg'], axis=1)

# Melt df
df_filtered = pd.melt(df_filtered, id_vars='year', var_name='country', value_name='value')
df_filtered['value'] = (df_filtered['value'] / 1000000).round()
df_filtered.columns = ['year','country', 'ba_year']

# Create the sum per year
df_sum_per_year = df_filtered.groupby('year').sum().reset_index()
df_sum_per_year.columns = ['year', 'ba_year_eu']

# Calculate the sum over all years per country
df_sum_per_country = df_filtered.groupby('country').sum().reset_index()
del df_sum_per_country['year']
df_sum_per_country.columns = ['country', 'ba_sum']
df_sum_per_country.to_csv('countries.csv')

# merge df
df_filtered = pd.merge(df_filtered, df_sum_per_country, how='left', on='country')
df_filtered = pd.merge(df_filtered, df_sum_per_year, how='left', on='year')

# Calculate the percentage of year based on sum of all years per country
df_filtered['ba_share_year'] = (df_filtered['ba_year'] / df_filtered['ba_sum']).round(2)

# join country avg. latitude
df_lat = pd.read_csv('countries_with_lat.csv')
df_filtered = pd.merge(df_filtered, df_lat, how='left', on='country')

df_filtered.head()

<ipython-input-12-549291f019d8>:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_sum_per_year = df_filtered.groupby('year').sum().reset_index()


,year,country,ba_year,ba_sum,ba_year_eu,ba_share_year,lat
0,2001,Albania,115.0,5553.0,24895.0,0.02,41.2
1,2002,Albania,22.0,5553.0,8380.0,0.00,41.2
2,2003,Albania,306.0,5553.0,21885.0,0.06,41.2
3,2004,Albania,46.0,5553.0,20225.0,0.01,41.2
4,2005,Albania,155.0,5553.0,15797.0,0.03,41.2


## Create Chart

In [ ]:
colors = ['#451104', '#7f2a0c', '#b93e04', '#e27711', '#f0af3c', '#fbfc93']

min_size = 5  # Set the minimum object size
max_size = 5000  # Set the maximum object size

chart = alt.Chart(df_filtered).mark_circle(opacity=0.9).encode(
    alt.X('year:N', axis=alt.Axis(
        labelExpr="datum.value === 2001 || datum.value === 2010 || datum.value === 2020 ? datum.value : ''",
        labelColor='orange', ticks=False, labelAngle=0
    )),
    alt.Y('country:N', axis=alt.Axis(labels=True, labelColor='orange', ticks=False), sort=alt.EncodingSortField(field='lat', order='descending')),
    size=alt.Size('ba_year:Q', scale=alt.Scale(domain=[df_filtered['ba_year'].min(), df_filtered['ba_year'].max()], range=[min_size, max_size])),
    color=alt.Color('ba_year_eu', scale=alt.Scale(domain=[df_filtered['ba_year_eu'].min(), df_filtered['ba_year_eu'].max()], range=colors), legend=None),
    tooltip=['ba_share_year', 'ba_year_eu']
).properties(
    width=800,
    height=800
).configure(
    padding={"left": 40, "top": 40, "right": 0, "bottom": 50},
    background='#000000'
).configure_axis(grid=False, domain=False, labelFont='Helvetica'
).configure_view(
    strokeWidth=0
).configure_axisY(
    titleAngle=0,
    titleY=0,
    titleX=-60,
    labelPadding=160,
    labelAlign='left'
).configure_axisX(
    labelPadding=20
)

chart


alt.Chart(...)

In [ ]:
colors = ['#451104', '#7f2a0c', '#b93e04', '#e27711', '#f0af3c', '#fbfc93']
custom_shape = 'M-15,-10 C0,-30 10,-30 15,-20 C20,-30 30,-30 45,-10 C30,-15 20,0 15,10 C10,0 0,0 -15,-10'


min_size = 1  # Set the minimum object size
max_size = 6000  # Set the maximum object size

chart = alt.Chart(df_filtered).mark_point(shape='diamond',filled=True, opacity=0.9).encode(
    alt.X('year:N', axis=alt.Axis(
        labelExpr="datum.value === 2001 || datum.value === 2010 || datum.value === 2020 ? datum.value : ''",
        labelColor='orange', ticks=False, labelAngle=0
    )),
    alt.Y('country:N', axis=alt.Axis(labels=True, labelColor='orange', ticks=False), sort=alt.EncodingSortField(field='lat', order='descending')),
    size=alt.Size('ba_year:Q', scale=alt.Scale(domain=[df_filtered['ba_year'].min(), df_filtered['ba_year'].max()], range=[min_size, max_size])),
    color=alt.Color('ba_year_eu', scale=alt.Scale(domain=[df_filtered['ba_year_eu'].min(), df_filtered['ba_year_eu'].max()], range=colors), legend=None),
    tooltip=['year','country','ba_share_year', 'ba_year_eu']
).properties(
    width=800,
    height=800
).configure(
    padding={"left": 40, "top": 40, "right": 0, "bottom": 50},
    background='#000000'
).configure_axis(grid=False, domain=False, labelFont='Helvetica'
).configure_view(
    strokeWidth=0
).configure_axisY(
    titleAngle=0,
    titleY=0,
    titleX=-60,
    labelPadding=160,
    labelAlign='left'
).configure_axisX(
    labelPadding=20
)

chart


alt.Chart(...)